In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import models
import torch
import numpy as np
import pandas as pd
import os
import nibabel as nib

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories and file paths
input_dir = "path_to_input_folder"
output_dir = "path_to_output_folder"
csv_file_path = os.path.join(output_dir, "data_labels.csv")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 1: Convert NIfTI to NumPy
for file_name in os.listdir(input_dir):
    if file_name.endswith(".nii.gz"):
        # Load the NIfTI file
        nii_file_path = os.path.join(input_dir, file_name)
        nii_data = nib.load(nii_file_path)
        numpy_array = nii_data.get_fdata()

        # Normalize the data
        numpy_array = (numpy_array - np.min(numpy_array)) / (np.max(numpy_array) - np.min(numpy_array))

        # Save the NumPy array
        output_file_name = os.path.splitext(os.path.splitext(file_name)[0])[0] + ".npy"
        npy_file_path = os.path.join(output_dir, output_file_name)
        np.save(npy_file_path, numpy_array)


In [ ]:
# Step 2: Define the custom Dataset class
class NpyDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]
        
        # Load the image and convert to tensor
        image = np.load(image_path)
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        label = torch.tensor(label, dtype=torch.long)

        return image, label


In [ ]:
# Step 3: Split dataset and initialize DataLoaders
dataset = NpyDataset(csv_file= csv_file_path)

train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
# Step 4: Define the model
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(model.fc.in_features, 2)  # Assuming binary classification
model = model.to(device)

In [ ]:
# Step 5: Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Step 6: Training loop
def train_model():
    model.train()
    for epoch in range(10):  # 10 epochs
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/10], Loss: {total_loss/len(train_loader):.4f}")



In [ ]:
# Step 7: Validation loop
def validate_model():
    model.eval()
    with torch.no_grad():
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Validation Loss: {total_loss/len(val_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")


In [ ]:
# Run the training and validation
train_model()
validate_model()